### CONFIGURATION

In [1]:
import os
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 200)
import numpy as np
import copy
import sys
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Hartwig imports
sys.path.append("/Users/peterpriestley/hmf/repos/scripts/analysisscripts") 
import analyseVCF as aVCF
import venn as vn

In [3]:
#VCF
VCF_PATH = "/Users/peterpriestley/hmf/analyses/ensembleRuleTesting/"

# BED
#BED_PATH = "/Users/peterpriestley/hmf/analyses/giabTruthsets/"
#BED_FILE_NAME = "NA12878_GIAB_highconf_IllFB-IllGATKHC-CG-Ion-Solid_ALLCHROM_v3.2.2_highconf.bed"
BED_PATH = "/Users/peterpriestley/hmf/analyses/slices/"
#BED_FILE_NAME = "cosmicCancerCensus.bed" 
BED_FILE_NAME = "CPCT_Slicing.bed"
USE_BED = True

# HEALTH CHECKS
HEALTH_CSV_PATH="/Users/peterpriestley/hmf/analyses/healthCheckAnalysis/"
HEALTH_CSV_FILENAME="healthCheckFile.csv"

In [4]:
# SPECIFY SAMPLES
VCF_SAMPLES=[]
for x in os.listdir(VCF_PATH):
    if x[-10:] == "sliced.vcf":
        VCF_SAMPLES.append(x)
print VCF_SAMPLES

#VCF_SAMPLES = ['160903_HMFregCPCT_FR10302737_FR10304566_CPCT02070012.filtered_variants_snpEff_snpSift_Cosmicv76_GoNLv5_sliced.vcf']

['160903_HMFregCPCT_FR10302737_FR10304566_CPCT02070012.filtered_variants_snpEff_snpSift_Cosmicv76_GoNLv5_sliced.vcf', '160903_HMFregCPCT_FR10302740_FR10304570_CPCT02050049.filtered_variants_snpEff_snpSift_Cosmicv76_GoNLv5_sliced.vcf', '160905_HMFregCPCT_FR10301943_FR12251966_CPCT02160001.filtered_variants_snpEff_snpSift_Cosmicv76_GoNLv5_sliced.vcf', '160905_HMFregCPCT_FR10302743_FR10304571_CPCT02050050.filtered_variants_snpEff_snpSift_Cosmicv76_GoNLv5_sliced.vcf', '160905_HMFregCPCT_FR12244549_FR12251919_CPCT02160002.filtered_variants_snpEff_snpSift_Cosmicv76_GoNLv5_sliced.vcf', '160905_HMFregCPCT_FR12244559_FR12251744_CPCT02160003.filtered_variants_snpEff_snpSift_Cosmicv76_GoNLv5_sliced.vcf', '160905_HMFregCPCT_FR12251688_FR10304573_CPCT02020348.filtered_variants_snpEff_snpSift_Cosmicv76_GoNLv5_sliced.vcf', '160905_HMFregCPCT_FR12254135_FR10304574_CPCT02020350.filtered_variants_snpEff_snpSift_Cosmicv76_GoNLv5_sliced.vcf', '160905_HMFregCPCT_FR12254668_FR12251934_CPCT02160004.filtered_

<h3> LOAD VCF

In [5]:
#Load all VCF_SAMPLES to DF
df = pd.DataFrame({})
for VCF_SAMPLE in VCF_SAMPLES:
    VCF_FILE_NAME = VCF_SAMPLE
    SAMPLE_NAMES = {VCF_SAMPLE[40:52]+'R':'normal',VCF_SAMPLE[40:52]+'T':'tumor'}
    if USE_BED:
        bed = aVCF.loadBEDFile(BED_PATH,BED_FILE_NAME)
        df = pd.concat([df,aVCF.loadVaraintsFromVCF(VCF_PATH,VCF_FILE_NAME,SAMPLE_NAMES,VCF_SAMPLE[40:52],True,True,bed,False)])
    else:
        df = pd.concat([df,aVCF.loadVaraintsFromVCF(VCF_PATH,VCF_FILE_NAME,SAMPLE_NAMES,VCF_SAMPLE[40:52],True)])

reading BED file
Bed File Loaded
reading vcf file: 160903_HMFregCPCT_FR10302737_FR10304566_CPCT02070012.filtered_variants_snpEff_snpSift_Cosmicv76_GoNLv5_sliced.vcf
Number variants loaded: 67
reading BED file
Bed File Loaded
reading vcf file: 160903_HMFregCPCT_FR10302740_FR10304570_CPCT02050049.filtered_variants_snpEff_snpSift_Cosmicv76_GoNLv5_sliced.vcf
Number variants loaded: 65
reading BED file
Bed File Loaded
reading vcf file: 160905_HMFregCPCT_FR10301943_FR12251966_CPCT02160001.filtered_variants_snpEff_snpSift_Cosmicv76_GoNLv5_sliced.vcf
Number variants loaded: 73
reading BED file
Bed File Loaded
reading vcf file: 160905_HMFregCPCT_FR10302743_FR10304571_CPCT02050050.filtered_variants_snpEff_snpSift_Cosmicv76_GoNLv5_sliced.vcf
Number variants loaded: 57
reading BED file
Bed File Loaded
reading vcf file: 160905_HMFregCPCT_FR12244549_FR12251919_CPCT02160002.filtered_variants_snpEff_snpSift_Cosmicv76_GoNLv5_sliced.vcf
Number variants loaded: 72
reading BED file
Bed File Loaded
reading

In [ ]:
#LOAD HEALTH_CHECKS
hc = pd.read_csv(HEALTH_CSV_PATH + HEALTH_CSV_FILENAME)
hc = hc[['SAMPLE','TUMOR_PERCENTAGE','TUMOR_TYPE','PIPELINE_VERSION']]
hc = hc[hc['SAMPLE'].str[12]=="T"]   # only tumor rows
hc['SAMPLE']=hc['SAMPLE'].str[:12]
hc = hc.set_index(['SAMPLE'])
df = pd.merge(df,hc,how='left', left_on='patientName',right_index=True)
df.fillna("Unknown", inplace=True)

<h3> High Level Stats

In [ ]:
pd.pivot_table(df[df.normalSGT!=df.tumorSGT], values='pos', index=['patientName'], columns=['annWorstImpact'], aggfunc='count',margins=True).fillna("")

In [ ]:
df[(df.patientName=='CPCT02010396')&(df.normalSGT!=df.tumorSGT)]

In [ ]:
df[['variantType','pos']].groupby(['variantType']).agg('count')

<h3> Allelic Depth

In [ ]:
#Alllele Freq By Caller
for columnName in list(df):
    if columnName.endswith('allele'):
        ser = df[df[columnName] != ''][columnName[:-6] + 'AF']
        ser = ser.sort_values()
        #ser[len(ser)] = ser.iloc[-1]
        cum_dist = np.linspace(0.,1.,len(ser))
        ser_cdf = pd.Series(cum_dist, index=ser,name=columnName[:-6]+": c="+str(ser.count())+" m="+str(round(ser.median(),2)))
        ser_cdf.plot(drawstyle='steps',legend=True,title=" Allelic Frequency by Caller",figsize=[15,6])